In [1]:
!pip install timm datasets[vision] keras tensorflow opencv-python-headless matplotlib scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 912.2 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 1.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 698.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 32.2 MB/s 

In [2]:
import timm
import torch
import datasets
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
from functools import partial
from sklearn.neighbors import NearestNeighbors
import pandas as pd

import tensorflow as tf
from keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import (load_img, img_to_array)
from tensorflow.keras.applications.vgg16 import (VGG16, preprocess_input)

2023-11-03 12:05:44.928714: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 12:05:44.977169: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 12:05:44.977204: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 12:05:44.977232: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 12:05:44.988333: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [3]:
model = timm.create_model('vgg16', pretrained=True, num_classes=0)

model.safetensors:   0%|          | 0.00/553M [00:00<?, ?B/s]

In [4]:
ds = datasets.load_from_disk('../datasets/test_train_new')

In [5]:
ds.features

{'image': Image(decode=True, id=None),
 'artist': ClassLabel(names=['Unknown Artist', 'boris-kustodiev', 'camille-pissarro', 'childe-hassam', 'claude-monet', 'edgar-degas', 'eugene-boudin', 'gustave-dore', 'ilya-repin', 'ivan-aivazovsky', 'ivan-shishkin', 'john-singer-sargent', 'marc-chagall', 'martiros-saryan', 'nicholas-roerich', 'pablo-picasso', 'paul-cezanne', 'pierre-auguste-renoir', 'pyotr-konchalovsky', 'raphael-kirchner', 'rembrandt', 'salvador-dali', 'vincent-van-gogh', 'hieronymus-bosch', 'leonardo-da-vinci', 'albrecht-durer', 'edouard-cortes', 'sam-francis', 'juan-gris', 'lucas-cranach-the-elder', 'paul-gauguin', 'konstantin-makovsky', 'egon-schiele', 'thomas-eakins', 'gustave-moreau', 'francisco-goya', 'edvard-munch', 'henri-matisse', 'fra-angelico', 'maxime-maufra', 'jan-matejko', 'mstislav-dobuzhinsky', 'alfred-sisley', 'mary-cassatt', 'gustave-loiseau', 'fernando-botero', 'zinaida-serebriakova', 'georges-seurat', 'isaac-levitan', 'joaquã\xadn-sorolla', 'jacek-malczewski'

In [17]:
label_class = ds.features["genre"]
label_mapping = {i: label_class.int2str(i) for i in range(ds.features["genre"].num_classes)}

In [44]:
label_mapping

{0: 'abstract_painting',
 1: 'cityscape',
 2: 'genre_painting',
 3: 'illustration',
 4: 'landscape',
 5: 'nude_painting',
 6: 'portrait',
 7: 'religious_painting',
 8: 'sketch_and_study',
 9: 'still_life',
 10: 'Unknown Genre'}

In [21]:
label_class.int2str(i) for i in range(ds.features["genre"].num_classes)


SyntaxError: invalid syntax (440808931.py, line 1)

In [31]:
num_classes = list(range(ds.features["genre"].num_classes))

In [32]:
num_classes

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [36]:
names = ds.features["genre"].names

In [57]:
label_class = ds.features["genre"]
num_classes = ds.features["genre"].num_classes

dict = {}

for i in range(num_classes):
    dict[i] = label_class.int2str(i)

In [59]:
dict

{0: 'abstract_painting',
 1: 'cityscape',
 2: 'genre_painting',
 3: 'illustration',
 4: 'landscape',
 5: 'nude_painting',
 6: 'portrait',
 7: 'religious_painting',
 8: 'sketch_and_study',
 9: 'still_life',
 10: 'Unknown Genre'}

In [41]:
label_class.int2str(1)

'cityscape'

In [37]:
class_labels = dict(zip(num_classes, names))

In [24]:
label_class

ClassLabel(names=['abstract_painting', 'cityscape', 'genre_painting', 'illustration', 'landscape', 'nude_painting', 'portrait', 'religious_painting', 'sketch_and_study', 'still_life', 'Unknown Genre'], id=None)

In [65]:
# convert hf dataset to tf dataset
tf_ds = ds.to_tf_dataset(
            columns=["eva_clip"], # the columns to be used as inputs to the model, X
            label_cols=["genre"], # columns containing class labels, y
            batch_size=2,
            shuffle=True
            )

In [9]:
def classfication_model(train_data, inp_size, hidden_layer_size, feature, embedding_col):
    
    num_classes = len(set(train_data[feature]))

    inp = Input(shape=(inp_size,))
    hidden_layer = Dense(hidden_layer_size, activation='relu')(inp)
    
    classification_layer = Dense(num_classes, activation='softmax')(hidden_layer)

    model = Model(inputs=inp, outputs=classification_layer)

    # maybe change loss function?
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
    
    model.fit(x=train_data[embedding_col], y=train_data[feature], steps_per_epoch = )


SyntaxError: invalid syntax (640361627.py, line 15)

In [66]:
inp = Input(shape=(1408,))
hidden_layer = Dense(500, activation='relu')(inp)
classification_layer = Dense(11, activation='softmax')(hidden_layer)
model = Model(inputs=inp, outputs=classification_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(tf_ds, epochs = 10)


Epoch 1/10
4/4 [==============================] - 1s 21ms/step - loss: 3.8825
Epoch 2/10
4/4 [==============================] - 0s 52ms/step - loss: 0.5237
Epoch 3/10
4/4 [==============================] - 0s 55ms/step - loss: 2.6673e-06
Epoch 4/10
4/4 [==============================] - 0s 53ms/step - loss: 4.8726e-06
Epoch 5/10
4/4 [==============================] - 0s 44ms/step - loss: 2.4644e-05
Epoch 6/10
4/4 [==============================] - 0s 49ms/step - loss: 4.0793e-05
Epoch 7/10
4/4 [==============================] - 0s 57ms/step - loss: 8.7127e-05
Epoch 8/10
4/4 [==============================] - 0s 53ms/step - loss: 1.5252e-04
Epoch 9/10
4/4 [==============================] - 0s 47ms/step - loss: 1.8992e-04
Epoch 10/10
4/4 [==============================] - 0s 41ms/step - loss: 1.7736e-04


In [67]:
ds_test = datasets.load_from_disk('../datasets/test_test_new')

In [68]:
# convert hf dataset to tf dataset
tf_ds_test = ds_test.to_tf_dataset(
            columns=["eva_clip"], # the columns to be used as inputs to the model, X
            label_cols=["genre"], # columns containing class labels, y
            batch_size=2,
            shuffle=True
            )

In [69]:
predictions = model.predict(ds_test['eva_clip'])

1/1 [==============================] - 0s 126ms/step


In [70]:
tf_predictions = model.predict(tf_ds_test)

1/1 [==============================] - 0s 155ms/step


In [28]:
predictions # returns the probability of each of the 11 classes

array([[6.9784485e-09, 1.1327097e-08, 1.8185634e-03, 1.4161369e-08,
        2.9237095e-02, 7.4296011e-08, 4.3137308e-12, 9.6893579e-01,
        3.5621991e-08, 2.3832699e-07, 8.1611824e-06],
       [1.9097739e-05, 5.1779061e-06, 5.0632912e-04, 1.4574429e-06,
        2.4428186e-03, 2.6425751e-05, 9.9169119e-07, 9.9694186e-01,
        3.3270965e-06, 3.1470725e-05, 2.1048381e-05]], dtype=float32)

In [29]:
tf_predictions == predictions

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True]])

In [35]:
predictions.shape

(2, 11)

In [72]:
predicted_classes = np.argmax(tf_predictions,axis=1)

In [24]:
from datasets import ClassLabel

hej = ClassLabel.int2sds['genre']tr(, predicted_classes)

AttributeError: 'list' object has no attribute 'num_classes'

In [75]:
from sklearn.metrics import classification_report
 
report = classification_report(ds_test['genre'],
                            predicted_classes, target_names = dict)

ValueError: Number of classes, 3, does not match size of target_names, 11. Try specifying the labels parameter

In [ ]:
    # create classification report from predicted and true labels
    report = classification_report(y_true,
                            predicted_classes, target_names = labels)
    
    # save report
    out_path = os.path.join("out", filename)

    with open(out_path, 'w') as file:
                file.write(report)

In [74]:
print(report)

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         1
           4       1.00      1.00      1.00         1
           7       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



In [34]:
ds_train.features['genre']

NameError: name 'ds_train' is not defined

In [45]:
ds.features['genre'].int2str()

AttributeError: 'Dataset' object has no attribute 'int2str'

In [43]:
ds_test[0]['style']

23

In [41]:
ds_test[1]['genre']

2

In [59]:
def compress_data(ds_train, feature_col, compressed_size, epochs):
    # change this !
    inp_shape = len(ds_train[0][feature_col])
    inp = Input(shape=(inp_shape,))
    compressed = Dense(compressed_size, activation = 'relu')(inp) # problem at bruge relu her ??? 
    out = Dense(inp_shape, activation = 'relu')(compressed)
    comp_model = Model(inputs=inp, outputs = out)

    comp_model.compile(optimizer='adam', loss='mse')
    comp_model.fit(ds_train[feature_col], ds_train[feature_col], epochs=epochs)
    encoder = Model(comp_model.input, comp_model.layers[-2].output)
    compressions = encoder.predict(ds_train[feature_col])

    compressed_list = compressions.tolist()
    new_ds = ds_train.add_column(f'{feature_col}_compressed', compressed_list)

    return new_ds

In [60]:
new_ds = compress_data(ds, 'eva_clip', 800, 10)

Epoch 1/10
1/1 [==============================] - 0s 356ms/step - loss: 1.4858
Epoch 2/10
1/1 [==============================] - 0s 25ms/step - loss: 1.0965
Epoch 3/10
1/1 [==============================] - 0s 22ms/step - loss: 1.0024
Epoch 4/10
1/1 [==============================] - 0s 14ms/step - loss: 0.9430
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.9007
Epoch 6/10
1/1 [==============================] - 0s 15ms/step - loss: 0.8742
Epoch 7/10
1/1 [==============================] - 0s 15ms/step - loss: 0.8530
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 0.8331
Epoch 9/10
1/1 [==============================] - 0s 22ms/step - loss: 0.8178
Epoch 10/10
1/1 [==============================] - 0s 79ms/step


/home/ucloud/.local/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


In [62]:
datasets.ClassLabel.int2str()

[[0.0,
  0.16740235686302185,
  0.0,
  0.0,
  0.0,
  1.21927809715271,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.2335799932479858,
  3.356360912322998,
  0.0,
  1.501168131828308,
  0.0,
  0.0,
  0.0,
  3.9061179161071777,
  0.0,
  2.958127498626709,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0819315910339355,
  0.0,
  3.975316047668457,
  0.0,
  0.0,
  0.883751630783081,
  0.0,
  0.045897163450717926,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.15666665136814117,
  0.0,
  0.0,
  0.9977874755859375,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.26855841279029846,
  0.0,
  0.0,
  1.0107312202453613,
  0.0,
  0.0,
  0.12577785551548004,
  1.1289780139923096,
  0.0,
  0.0,
  0.0,
  0.658117413520813,
  0.0,
  0.0,
  0.0,
  0.0,
  1.2330368757247925,
  2.10953688621521,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.123209834098816,
  1.578292965888977,
  1.394795536994934,
  0.0,
  0.0,
  2.621147394180298,
  0.0,
  0.0,
  0.0,
  0.4265094995498657,
  0

In [52]:
listen = compressed_features.tolist()

In [55]:
len(listen[0])

800

In [18]:
len(ds)

8